## Introduction

A huge part of data science involves acquiring raw data and getting it into a form ready for analysis. Some have estimated that data scientists spend 80% of their time cleaning and manipulating data, and only 20% of their time actually analyzing it or building models from it.

When we receive raw data, we have to do a number of things before we’re ready to analyze it, possibly including:

1. diagnosing the “tidiness” of the data — how much data cleaning we will have to do
2. reshaping the data — getting right rows and columns for effective analysis
3. combining multiple files
4. changing the types of values — how we fix a column where numerical values are stored as strings, for example
5. dropping or filling missing values - how we deal with data that is incomplete or missing
6. manipulating strings to represent the data better

## Diagnose the Data

We often describe data that is easy to analyze and visualize as “tidy data”. What does it mean to have tidy data?

For data to be tidy, it must have:

1. Each variable as a separate column
2. Each row as a separate observation

The first step of diagnosing whether or not a dataset is tidy is using pandas functions to explore and probe the dataset.

You’ve seen most of the functions we often use to diagnose a dataset for cleaning. Some of the most useful ones are:

1. .head() — display the first 5 rows of the table
2. .info() — display a summary of the table
3. .describe() — display the summary statistics of the table
4. .columns — display the column names of the table
5. .value_counts() — display the distinct values for a column

## Dealing with Multiple Files

Often, you have the same data separated out into multiple files.

Let’s say that we have a ton of files following the filename structure: 'file1.csv', 'file2.csv', 'file3.csv', and so on. The power of pandas is mainly in being able to manipulate large amounts of structured data, so we want to be able to get all of the relevant information into one table so that we can analyze the aggregate data.

We can combine the use of glob, a Python library for working with files, with pandas to organize this data better. glob can open multiple files by using regex matching to get the filenames:

import glob

files = glob.glob("file*.csv")

df_list = []

for filename in files:

    data = pd.read_csv(filename)

    df_list.append(data)

df = pd.concat(df_list)

print(files)

This code goes through any file that starts with 'file' and has an extension of .csv. It opens each file, reads the data into a DataFrame, and then concatenates all of those DataFrames together.

![mf1](https://i.imgur.com/TIU8L55.jpg)

![mf2](https://i.imgur.com/6ONqa8B.jpg)

## Reshaping your Data

Since we want

1. Each variable as a separate column
2. Each row as a separate observation

We can use pd.melt() to do this transformation. .melt() takes in a DataFrame, and the columns to unpack.

The parameters you provide are:

1. frame: the DataFrame you want to melt
2. id_vars: the column(s) of the old DataFrame to preserve
3. value_vars: the column(s) of the old DataFrame that you want to turn into variables
4. value_name: what to call the column of the new DataFrame that stores the values
5. var_name: what to call the column of the new DataFrame that stores the variables

The default names may work in certain situations, but it’s best to always have data that is self-explanatory. Thus, we often use .columns() to rename the columns after melting.

![rsd](https://i.imgur.com/otV0Hcx.jpg)

## Dealing with Duplicates

Often we see duplicated rows of data in the DataFrames we are working with. This could happen due to errors in data collection or in saving and loading the data.

To check for duplicates, we can use the pandas function .duplicated(), which will return a Series telling us which rows are duplicate rows.

We can use the pandas .drop_duplicates() function to remove all rows that are duplicates of another row.

If we wanted to remove every row with a duplicate value in the item column, we could specify a subset:

fruits = fruits.drop_duplicates(subset=['item'])

Make sure that the columns you drop duplicates from are specifically the ones where duplicates don’t belong. You wouldn’t want to drop duplicates with the price column as a subset, for example, because it’s okay if multiple items cost the same amount!

![dup](https://i.imgur.com/mlQDWc7.jpg)

## Splitting by Index

In trying to get clean data, we want to make sure each column represents one type of measurement. Often, multiple measurements are recorded in the same column, and we want to separate these out so that we can do individual analysis on each variable.

Let’s say we have a column “birthday” with data formatted in MMDDYYYY format. In other words, “11011993” represents a birthday of November 1, 1993. We want to split this data into day, month, and year so that we can use these columns as separate features.

In this case, we know the exact structure of these strings. The first two characters will always correspond to the month, the second two to the day, and the rest of the string will always correspond to year. We can easily break the data into three separate columns by splitting the strings using .str:

df['month'] = df.birthday.str[0:2] # Create the 'month' column

df['day'] = df.birthday.str[2:4] # Create the 'day' column

df['year'] = df.birthday.str[4:] # Create the 'year' column

The first command takes the first two characters of each value in the birthday column and puts it into a month column. The second command takes the second two characters of each value in the birthday column and puts it into a day column. The third command takes the rest of each value in the birthday column and puts it into a year column.

![si1](https://i.imgur.com/jwFnP9X.jpg)

![si2](https://i.imgur.com/Agx4f4y.jpg)

## Splitting by Character

Let’s say we have a column called “type” with data entries in the format "admin_US" or "user_Kenya". Just like we saw before, this column actually contains two types of data. One seems to be the user type (with values like “admin” or “user”) and one seems to be the country this user is in (with values like “US” or “Kenya”).

We can no longer just split along the first 4 characters because admin and user are of different lengths. Instead, we know that we want to split along the "_". Using that, we can split this column into two separate, cleaner columns:

df['str_split'] = df.type.str.split('_') # Create the 'str_split' column

df['usertype'] = df.str_split.str.get(0) # Create the 'usertype' column

df['country'] = df.str_split.str.get(1) # Create the 'country' column

![sc1](https://i.imgur.com/QnFon8G.jpg)

## Looking at Types

Each column of a DataFrame can hold items of the same data type or dtype. The dtypes that pandas uses are: float, int, bool, datetime, timedelta, category and object. Often, we want to convert between types so that we can do better analysis. If a numerical category like "num_users" is stored as a Series of objects instead of ints, for example, it makes it more difficult to do something like make a line graph of users over time.

To see the types of each column of a DataFrame, we can use:

print(df.dtypes)

![t1](https://i.imgur.com/p2g3LfY.jpg)

## String Parsing

Sometimes we need to modify strings in our DataFrames to help us transform them into more meaningful metrics. 

The 'price' column is actually composed of strings representing dollar amounts. This column could be much better represented in floats, so that we could take the mean, calculate other aggregate statistics, or compare different fruits to one another in terms of price.

First, we can use what we know of regex to get rid of all of the dollar signs:

fruit.price = fruit['price'].replace('[\$,]', '', regex=True)

Then, we can use the pandas function .to_numeric() to convert strings containing numerical values to integers or floats:

fruit.price = pd.to_numeric(fruit.price)

![sp1](https://i.imgur.com/vonR1M8.jpg)

## More String Parsing

Sometimes we want to do analysis on numbers that are hidden within string values. We can use regex to extract this numerical data from the strings they are trapped in.

To extract the numbers from the string we can use pandas’ .str.split() function:

split_df = df['exerciseDescription'].str.split('(\d+)', expand=True)

Then, we can assign columns from this DataFrame to the original df:

df.reps = pd.to_numeric(split_df[1])

df.exercise = split_df[2].replace('[\- ]', '', regex=True)

![sp2](https://i.imgur.com/8Hz9CF5.jpg)

![sp3](https://i.imgur.com/T5yc3na.jpg)

## Missing Values

We often have data with missing elements, as a result of a problem with the data collection process or errors in the way the data was stored. The missing elements normally show up as NaN (or Not a Number) values. 

### Method 1: drop all of the rows with a missing value

We can use .dropna() to do this:

bill_df = bill_df.dropna()

This command will result in the DataFrame without the incomplete rows.

### Method 2: fill the missing values with the mean of the column, or with some other aggregate value.

We can use .fillna() to do this:

bill_df = bill_df.fillna(value={"bill":bill_df.bill.mean(), "num_guests":bill_df.num_guests.mean()})

This command will result in the DataFrame with the respective mean of the column in the place of the original NaNs.

![mv1](https://i.imgur.com/Qg1qF4N.jpg)

## Review

We have looked at a number of different methods we may use to get data into the format we want for analysis.

Specifically, we have covered:

1. diagnosing the “tidiness” of the data
2. reshaping the data
3. combining multiple files
4. changing the types of values
5. dropping or filling missing values - how we deal with data that is incomplete or missing
6. manipulating strings to represent the data better

You can use these methods to transform your datasets to be clean and easy to work with!

## Quiz

![qd1](https://i.imgur.com/T4ekNxk.jpg)

![qd2](https://i.imgur.com/AuHAtH0.jpg)

![qd3](https://i.imgur.com/UbDxMws.jpg)

![qd4](https://i.imgur.com/5BK0t9G.jpg)

![qd5](https://i.imgur.com/nn2mAVt.jpg)

![qd6](https://i.imgur.com/n3gGb9p.jpg)

![qd7](https://i.imgur.com/OvSsG0Z.jpg)